# Laborarbeit Künstliche Intelligenz

## Thema :  Bayes Netze und Case Based Reasoning

### Namen der Studierenden: 

(Hinweis: Es sind Namen anzugeben und keine Matrikelnummern. Matrikelnummern werden ausschließlich bei Klausuren zur Anonymisierung verwendet)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Einlesen der CSV-Datei
file_path = './bikedata/sampled_data_<ihre_Nummer>.csv'   # Einsetzen!
data = pd.read_csv(file_path, sep=";", decimal=",")

#### Etwas Analyse vorab ... gerne erweitern

In [ ]:

# Numerische Attribute - einzelne Boxplots
numeric_columns = data.select_dtypes(include='number').columns

for column in numeric_columns:
    plt.figure(figsize=(8, 6))
    sns.boxplot(y=data[column])
    plt.title(f'Boxplot der numerischen Attributs: {column}')
    plt.ylabel(column)
    plt.grid(True)
    plt.show()

# Kategoriale Attribute - Häufigkeitsverteilungen
categorical_columns = data.select_dtypes(include='object').columns

for column in categorical_columns:
    plt.figure(figsize=(10, 5))
    data[column].value_counts().plot(kind='bar')
    plt.title(f'Häufigkeitsverteilung von {column}')
    plt.xlabel(column)
    plt.ylabel('Häufigkeit')
    plt.grid(True)
    plt.show()


In [ ]:
# Scatterplot für Elevation Gain und Distance basierend auf Bike Type

plt.figure(figsize=(10, 6))
colors = {'race bike': 'red', 'mtb': 'green', 'trecking bike': 'yellow'}
sns.scatterplot(data=data, x='Elevation Gain', y='Distance', hue='Bike Type', palette=colors)
plt.title('Scatterplot von Elevation Gain vs Distance für Bike Type')
plt.xlabel('Elevation Gain')
plt.ylabel('Distance')
plt.grid(True)
plt.show()

In [ ]:
# Boxplots für Elevation Gain und Distance

# Boxplot für Elevation Gain
plt.figure(figsize=(12, 6))
sns.boxplot(x='Bike Type', y='Elevation Gain', data=data, order=['race bike', 'mtb', 'trecking bike'])
plt.title('Boxplot von Elevation Gain für Bike Types')
plt.xlabel('Bike Type')
plt.ylabel('Elevation Gain')
plt.grid(True)
plt.show()

# Boxplot für Distance
plt.figure(figsize=(12, 6))
sns.boxplot(x='Bike Type', y='Distance', data=data, order=['race bike', 'mtb', 'trecking bike'])
plt.title('Boxplot von Distance für Bike Types')
plt.xlabel('Bike Type')
plt.ylabel('Distance')
plt.grid(True)
plt.show()

## Aufgabenteil 1: Bayes Netze
Entwerfen Sie ein KI Modell auf Basis der Bayes Netze und setzen Sie dieses
als Jupyter Notebook um. Insbesondere wird eine geeignete Netzarchitektur und ein
Vorschlag für eine plausible Wahrscheinlichkeitsverteilung benötigt.

In [ ]:
# !pip install pgmpy 

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD


### 1. Modell anlegen

Dies enthält zunächst die (kausalen) Abhängigkeiten.

In [ ]:

# Modell ersetzen durch eigenes für diese Aufgabe

# (Beispiel : Emotionserkennung - aus Übungslabor)

model = BayesianNetwork([('Angst','Sprechgeschwindigkeit'), 
                         ('Ekel','Sprechgeschwindigkeit' ),
                         ('Wut','Sprechgeschwindigkeit' ),
                         ('Freude','Sprechgeschwindigkeit'),
                         ('Traurigkeit','Sprechgeschwindigkeit')])

### Begründung zum Modell

<...Ihr Text...>

### 2. CPTs anlegen

Das ist der Kern des Bayes'schen Netzes. Die CPTs können aus Daten gewonnen werden, sofern viele da sind. Ansonsten ist zusätzlich Hintergrundwissen und Schätzung zu verwenden.

In [ ]:
# Beispiel aus dem Übungslabor - ersetzen Sie dies durch eigene Berechnungen für die CPTs

import numpy as np

#logisch = 1. wahrscheinlichkeit ... wählbar.
x = 1.0

values = []

# Durchlaufe alle 32 Kombinationen der Eltern
for tra in [0, 1]:  # Traurigkeit: 0 = 'nein', 1 = 'ja'
    for ang in [0, 1]:  # Angst: 0 = 'nein', 1 = 'ja'
        for ek in [0, 1]:  # Ekel: 0 = 'nein', 1 = 'ja'
            for wu in [0, 1]:  # Wut: 0 = 'nein', 1 = 'ja'
                for fr in [0, 1]:  # Freude: 0 = 'nein', 1 = 'ja'
                    if tra == 1:  # Wenn Traurigkeit = 'ja'
                        # Wenn Traurigkeit gegeben ist, wird 'langsam' auf x und 'schnell' auf 1-x gesetzt
                        values.append([1-x, x]) 
                    else:  # Wenn Traurigkeit = 'nein'
                        # Wenn Angst, Ekel, Wut oder Freude gegeben ist, wird 'schnell' auf x und 'langsam' auf 1-x gesetzt
                        if ang == 1 or ek == 1 or wu == 1 or fr == 1:
                            values.append([x, 1-x])  # [langsam=1-x, schnell=x]
                        else:
                            values.append([0.5, 0.5])  # Unwissenheit -> beide 0.5

values = np.array(values).T

# CPD für Sprechgeschwindigkeit erstellen
cpd_Sprechgeschwindigkeit = TabularCPD(
    variable='Sprechgeschwindigkeit', variable_card=2,
    values=values,  # Wahrscheinlichkeiten für jede Kombination von Eltern
    evidence=['Traurigkeit', 'Angst', 'Ekel', 'Wut', 'Freude'],
    evidence_card=[2, 2, 2, 2, 2],
    state_names={'Sprechgeschwindigkeit':['schnell','langsam'],
                 'Angst': ['nein','ja'],
                 'Ekel': ['nein','ja'],
                 'Wut': ['nein','ja'],
                 'Freude': ['nein','ja'],
                 'Traurigkeit': ['nein','ja']
                })

model.add_cpds(cpd_Sprechgeschwindigkeit, cpd_Angst, cpd_Wut, cpd_Freude, cpd_Ekel, cpd_Traurigkeit)

model.check_model()

### Begründung zur Füllung der CPTs

<...Ihr Text...>

### 3. Inferenz

Nutzen Sie nun die Inferenz der Bayes Netze um die Klassifikation durchzuführen.


In [ ]:
# Beispielcode aus dem Übungslabor - ersetzen Sie dies durch geeigneten eigenen (umfangreicheren) Code

from pgmpy.inference import VariableElimination

infer = VariableElimination(model)

distrib = infer.query(['Angst'])
print(distrib)

### 4. Test und Bewertung 

Testen Sie die Funktionsweise Ihres Netzes und diskutieren Sie die Ergebnisse.

In [ ]:
# Ihr Code



### Diskussion der Ergebnisse 

<...Ihr Text...>

## Aufgabenteil 2: Case Based Reasoning

Entwerfen Sie ein KI Modell auf Basis von Case Based Reasoning und setzen
Sie dieses als Jupyter Notebook um. Wählen Sie geeignete Vorgehensweisen um die Fallbasis
aufzubauen (CBL) und geeignete Ähnlichkeitsmaße.

### 1. Ähnlichkeitsmaß definieren

Entwerfen Sie ein zur Aufgabenstellung passendes und geeignetes Ähnlichkeitsmaß.

In [ ]:
# Beispiel - hier soll Ihr Code eingefügt werden

def aehnlichkeit(person1, person2):
    """
    Berechnet die Ähnlichkeit zwischen zwei Personen.

    Args:
        person1: Eine Zeile des Datensatzes als Dictionary.
        person2: Eine Zeile des Datensatzes als Dictionary.

    Returns:
        float: Ein Wert zwischen 0 und 1, der die Ähnlichkeit angibt.
    """

    aehnlichkeit = 1.0

    return aehnlichkeit

### Begründung zur Wahl des Ähnlichkeitsmaßes

<...Ihr Text...>

### 2. CBR Zyklus umsetzen

Zunächst wird RETRIEVE benötigt. Dann sollte geprüft werden ob das Ergebnis bereits ausreicht. Wenn nicht, sollte geprüft werden, ob man die Daten in die Fallbasis aufnimmt. Die Fallbasis ist klein zu halten.

In [ ]:
# Beispiel aus dem Übungslabor - durch eigene Lösung zu ersetzen

def finde_aehnlichsten_fall(df, person_zu_vergleichen):
    """
    Findet die Person im DataFrame, die die größte Ähnlichkeit zur gegebenen Person hat.

    Args:
        df: Ein Pandas DataFrame mit den Personendaten.
        person_zu_vergleichen: Ein Dictionary, das die Merkmale der Person zum Vergleich enthält.

    Returns:
        tuple: Ein Tupel mit (index, ähnlichkeit), wobei index der Index der ähnlichsten Person im DataFrame und ähnlichkeit der Ähnlichkeitswert ist.
    """

    max_aehnlichkeit = -1
    index_aehlichste_person = -1

    for i, row in df.iterrows():
        person_im_df = row.to_dict()

        a = aehnlichkeit(person_zu_vergleichen, person_im_df)

        if a > max_aehnlichkeit:
            max_aehnlichkeit = a
            index_aehlichste_person = i
            aehnlichster_datensatz = row

    return index_aehlichste_person, max_aehnlichkeit, aehnlichster_datensatz

In [ ]:
# CBL umsetzen (vgl. Übungslabor)



### Begründungen 

<...Ihr Text...>

### 3. Test und Bewertung

Wie gut ist der Klassifikator? Prüfen Sie Ihre Konfiguration (u.a. abhängig von Ähnlichkeitsfunktion) und testen Sie geeignet. 

In [ ]:
# Ihr Code


### Diskussion der Ergebnisse 

<...Ihr Text...>

# Abschluss

Vergleich der beiden Verfahren. Ggf. ist hier noch Code zum Vergleich zu ergänzen, ansonsten weitgehend durch Text.

<...Ihr Text...>